In [2]:
# 1) Enable third-party widget support in Colab
from google.colab import output
output.enable_custom_widget_manager()

# 2) Imports
import time, random, requests
import numpy as np
import plotly.graph_objs as go
from IPython.display import display
from sklearn.ensemble import IsolationForest
from collections import deque

# 3) Configuration
TELEGRAM_BOT_TOKEN   = '8058108604:AAFeVgtMm5KOALY8anNihv2vTmHUQbK7b2g'
TELEGRAM_CHAT_ID     = '825745351'

MAX_WINDOW           = 120    # Number of points shown on graph
ANOMALY_INTERVAL     = 30     # Inject anomaly every 30 seconds
STREAM_DELAY         = 0.5    # Half-second between readings
TOTAL_ITERATIONS     = None   # Set to an int to stop automatically

# 4) Telegram alert function
def send_telegram_alert(t, p, v):
    msg = f"Anomaly!\nTemp: {t}°C\nPressure: {p} bar\nVibration: {v} Hz"
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    requests.post(url, data={'chat_id': TELEGRAM_CHAT_ID, 'text': msg})

# 5) Train Isolation Forest on simulated normal data
def train_model(n_samples=1000):
    X = np.array([
        [random.normalvariate(75,3),
         random.normalvariate(30,2),
         random.normalvariate(10,1)]
        for _ in range(n_samples)
    ])
    m = IsolationForest(contamination=0.01)
    m.fit(X)
    return m

# 6) Simulation + live chart
def run_simulation():
    model = train_model()

    # Buffers
    ts = deque(maxlen=MAX_WINDOW)
    T  = deque(maxlen=MAX_WINDOW)
    P  = deque(maxlen=MAX_WINDOW)
    V  = deque(maxlen=MAX_WINDOW)
    A  = deque(maxlen=MAX_WINDOW)

    # Create FigureWidget with four traces
    fig = go.FigureWidget(
        data=[
            go.Scatter(name='Temperature', x=[], y=[], mode='lines+markers', line=dict(color='blue')),
            go.Scatter(name='Pressure',    x=[], y=[], mode='lines+markers', line=dict(color='green')),
            go.Scatter(name='Vibration',   x=[], y=[], mode='lines+markers', line=dict(color='orange')),
            go.Scatter(name='Anomaly',     x=[], y=[], mode='markers',       marker=dict(color='red', size=10, symbol='x'))
        ],
        layout=go.Layout(
            title="Real-Time IoT Sensor Dashboard",
            xaxis_title="Time Step",
            yaxis_title="Sensor Values",
            template="plotly_white",
            height=500
        )
    )
    display(fig)

    start_time = time.time()
    next_anom = ANOMALY_INTERVAL
    i = 0

    try:
        while TOTAL_ITERATIONS is None or i < TOTAL_ITERATIONS:
            elapsed = int(time.time() - start_time)

            # Generate sensor readings
            temp      = round(random.normalvariate(75,3),2)
            pressure  = round(random.normalvariate(30,2),2)
            vibration = round(random.normalvariate(10,1),2)

            # Inject anomaly every 30 seconds
            if elapsed >= next_anom:
                temp      += random.uniform(15,25)
                pressure  -= random.uniform(5,10)
                vibration += random.uniform(10,20)
                next_anom += ANOMALY_INTERVAL

            # Detect anomaly
            is_anom = int(model.predict([[temp, pressure, vibration]])[0] == -1)

            # Append to buffers
            ts.append(i)
            T.append(temp)
            P.append(pressure)
            V.append(vibration)
            A.append(temp if is_anom else np.nan)

            # Update traces in place
            fig.data[0].x, fig.data[0].y = list(ts), list(T)
            fig.data[1].x, fig.data[1].y = list(ts), list(P)
            fig.data[2].x, fig.data[2].y = list(ts), list(V)
            fig.data[3].x, fig.data[3].y = list(ts), list(A)

            # Send alert if anomaly
            if is_anom:
                print(f"Anomaly @ step {i}: Temp={temp}, Pressure={pressure}, Vibration={vibration}")
                send_telegram_alert(temp, pressure, vibration)

            time.sleep(STREAM_DELAY)
            i += 1

    except KeyboardInterrupt:
        print(f"\nSimulation stopped after {i} steps.")

# 7) Start
run_simulation()


FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines+markers',
              'name': 'Temperature',
              'type': 'scatter',
              'uid': '7b1f0c2c-6af0-45cd-91df-b1d3e68c7659',
              'x': [],
              'y': []},
             {'line': {'color': 'green'},
              'mode': 'lines+markers',
              'name': 'Pressure',
              'type': 'scatter',
              'uid': '48559f5f-dcdb-4243-915e-eb77aef6773e',
              'x': [],
              'y': []},
             {'line': {'color': 'orange'},
              'mode': 'lines+markers',
              'name': 'Vibration',
              'type': 'scatter',
              'uid': '4c369ca9-968e-4dcb-a3c6-501ceabe6860',
              'x': [],
              'y': []},
             {'marker': {'color': 'red', 'size': 10, 'symbol': 'x'},
              'mode': 'markers',
              'name': 'Anomaly',
              'type': 'scatter',
              'uid': '65154532-1ddd-4d14-8647

Anomaly @ step 57: Temp=93.14539867901645, Pressure=24.755728900223104, Vibration=24.272712470119053
Anomaly @ step 113: Temp=98.3962877543834, Pressure=23.687645106887903, Vibration=21.0742925878448
Anomaly @ step 155: Temp=82.43, Pressure=26.37, Vibration=11.01
Anomaly @ step 168: Temp=98.3867667176127, Pressure=19.392565372244785, Vibration=22.86649314956402
Anomaly @ step 193: Temp=70.12, Pressure=28.3, Vibration=12.39
Anomaly @ step 223: Temp=92.45812124495009, Pressure=23.29879036864967, Vibration=22.328997452860015

Simulation stopped after 230 steps.
